# convolve with numba

Demonstration on fast image processing from Computerphile https://www.youtube.com/watch?v=SiJpkucGa1o&t=0s.
Replaced cython with numba Just-in-Time compiler.

In [1]:
import numpy as np
from PIL import Image
from numba import njit

Helper functions to read and write an image into and from a CxHxW numpy float32 array 

In [2]:
def read_image(path):
    img = np.array(Image.open(path))
    arr = np.ascontiguousarray(img.transpose(2,0,1), dtype=np.float32)
    arr /= 255
    return arr

def write_image(img, path):
    img *= 255
    img = img.transpose(1,2,0).astype(np.uint8)
    Image.fromarray(img).save(path)

Generate gaussian kernels

In [3]:
# Produces a 2D gaussian kernel of standard deviation sigma and size 2*sigma+1
def gaussian_kernel_2d(sigma):
    kernel_radius = np.ceil(sigma) * 3
    kernel_size = kernel_radius * 2 + 1
    ax = np.arange(-kernel_radius, kernel_radius + 1., dtype=np.float32)
    xx, yy = np.meshgrid(ax, ax)
    kernel = np.exp(-(xx**2 + yy**2) / (2. * sigma**2))
    return kernel / np.sum(kernel)

# Produces a 1D gaussian kernel of standard deviation sigma and size 2*sigma+1
def gaussian_kernel_1d(sigma):
    kernel_radius = np.ceil(sigma) * 3
    kernel_size = kernel_radius * 2 + 1
    ax = np.arange(-kernel_radius, kernel_radius + 1., dtype=np.float32)
    kernel = np.exp(-(ax**2) / (2. * sigma**2))
    return (kernel / np.sum(kernel)).reshape(1,kernel.shape[0])

convolve function just in time compiled with numba

In [4]:
@njit()
def convolve(image, output, kernel):
    channel_count = image.shape[0]
    image_height = image.shape[1]
    image_width = image.shape[2]

    kernel_height = kernel.shape[0]
    kernel_halfh = kernel_height // 2
    kernel_width = kernel.shape[1]
    kernel_halfw = kernel_width // 2
    
    # Do convolution
    for x in range(image_width):
        for y in range(image_height):
            # Calculate usable image / kernel range
            x_min = max(0, x - kernel_halfw)
            x_max = min(image_width - 1, x + kernel_halfw)
            y_min = max(0, y - kernel_halfh)
            y_max = min(image_height - 1, y + kernel_halfh)

            # Convolve filter
            for c in range(channel_count):
                value = 0
                total = 0
                for u in range(x_min, x_max + 1):
                    for v in range(y_min, y_max + 1):
                        tmp = kernel[v - y + kernel_halfh, u - x + kernel_halfw]
                        value += image[c, v, u] * tmp  
                        total += tmp
                output[c, y, x] = value / total

In [5]:
def main(input_file, output_file, sigma, seperate_filters):

    # Read image - I'm using floats to store the image, this isn't necessary but saves casting etc. during convolution
    img = read_image(input_file)

    # Create output of the same size
    output = np.zeros(img.shape, dtype=np.float32)


    if not seperate_filters:
        # NxN convolution
        kernel_2d = gaussian_kernel_2d(sigma) # You could create your own kernel here!

        # Convolve
        convolve(img, output, kernel_2d)

    else:
        # Nx1 -> 1xN convolution
        kernel_1d = gaussian_kernel_1d(sigma)

        # We need to store the half convolved intermediate image.
        # You could save time by going img -> output-> img and not allocating this array.
        # Bearing in mind if you do this you can't use img for anything else.
        intermediate = np.zeros(img.shape, dtype=np.float32) 

        # Convolve in two passes - we must store and use the intermediate image, don't read from the input both times!
        convolve(img, intermediate, kernel_1d)
        convolve(intermediate, output, kernel_1d.transpose())

    # Save final image
    write_image(output, output_file)

## Input Image
![Input Image](Androids.jpg)

In [6]:
%%timeit
main('androids.jpg', 'output.jpg', 10, False)

14.6 s ± 784 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
main('androids.jpg', 'output.jpg', 10, True)

840 ms ± 86.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Output Image
![Input Image](output.jpg)